## STAGE 1 TASK 3 - READING AND MERGING OF ENRICHMENT DATA(Hospital Beds)
#### Member: Ankitha Chavan
#### Reading the cleaning Enchriment data and merging it with covid data. Here i have used the covid data from super_covid file which i have saved in the super_covid notebook.

In [1]:
import pandas as pd 
covid_data = pd.read_csv(r"..\data\super_covid.csv", encoding = 'unicode_escape')
hospital_data = pd.read_csv(r"..\data\COVID-19_Reported_Patient_Impact_and_Hospital_Capacity_by_Facility.csv", encoding = 'unicode_escape')

In [2]:
#Dropping the first column as the index are repeating.
del covid_data[covid_data.columns[0]]
#converting to datetime format
covid_data["Date"] = pd.to_datetime(covid_data["Date"])
#analysis of data from start of 2021 january to 2021 december end.
covid_data_2021 = covid_data[(covid_data["Date"] >= '2021-01-01') & (covid_data["Date"] < '2022-01-01')]
#Sorting the vales and resetting the index from 0
covid_data_2021= covid_data_2021.sort_values(by=['Date']).reset_index(drop=True)
covid_data_2021

,countyFIPS,County Name,State,StateFIPS,Date,Number of Cases,Number of Deaths,population
0,1001,Autauga County,AL,1,2021-01-01,4239,50,55869
1,18065,Henry County,IN,18,2021-01-01,4310,55,47972
2,39117,Morrow County,OH,39,2021-01-01,1834,4,35328
3,21137,Lincoln County,KY,21,2021-01-01,1443,23,24549
4,12103,Pinellas County,FL,12,2021-01-01,44975,1049,974996
...,...,...,...,...,...,...,...,...
1146825,39083,Knox County,OH,39,2021-12-31,9906,189,62322
1146826,39081,Jefferson County,OH,39,2021-12-31,10833,251,65325
1146827,39079,Jackson County,OH,39,2021-12-31,6371,110,32413
1146828,39123,Ottawa County,OH,39,2021-12-31,6527,114,40525


In [3]:
#In covid date its daily and hospital data is weekly so converting the covid data to weekly.
covid_data_2021['Date'] = pd.to_datetime(covid_data_2021['Date']) - pd.to_timedelta(7, unit='d')
covid_data_2021_weekly = covid_data_2021.groupby(['countyFIPS','County Name','StateFIPS','State','population', 
                                                  pd.Grouper(key='Date', freq='W-FRI')])['Number of Cases','Number of Deaths'].sum().sort_values('Date').reset_index()
covid_data_2021_weekly

C:\Users\ankit\AppData\Local\Temp\ipykernel_13628\868179570.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  covid_data_2021_weekly = covid_data_2021.groupby(['countyFIPS','County Name','StateFIPS','State','population',


,countyFIPS,County Name,StateFIPS,State,population,Date,Number of Cases,Number of Deaths
0,1001,Autauga County,1,AL,55869,2020-12-25,4239,50
1,27027,Clay County,27,MN,64222,2020-12-25,6163,78
2,27029,Clearwater County,27,MN,8818,2020-12-25,657,14
3,27031,Cook County,27,MN,5463,2020-12-25,104,0
4,48503,Young County,48,TX,18010,2020-12-25,1425,32
...,...,...,...,...,...,...,...,...
166521,39015,Brown County,39,OH,43432,2021-12-24,55465,932
166522,39013,Belmont County,39,OH,67006,2021-12-24,78853,1707
166523,39011,Auglaize County,39,OH,45656,2021-12-24,60207,927
166524,39151,Stark County,39,OH,370606,2021-12-24,427539,8946


In [4]:
#converting fips_code colum to integer datatype and renaming the column.
hospital_data = hospital_data[hospital_data['fips_code'].notna()]
hospital_data['fips_code'] = hospital_data['fips_code'].astype(int)
hospital_data = hospital_data.rename(columns={'fips_code': 'countyFIPS'}) 

#converting zip column to integer datatype
hospital_data = hospital_data[hospital_data['zip'].notna()]
hospital_data['zip'] = hospital_data['zip'].astype(int)

#converting collection week column in hospital data to date and converting to datatime format.
hospital_data = hospital_data.rename(columns={'collection_week': 'Date'}) 
hospital_data['Date'] =  pd.to_datetime(hospital_data['Date'])

##analysis of data from start of 2021 january to 2021 december end.
hospital_data_2021 = hospital_data[(hospital_data["Date"] >= '2021-01-01') & (hospital_data["Date"] < '2022-01-01')]
#Resetting the index.
hospital_data_2021 = hospital_data_2021.sort_values(by=['Date']).reset_index(drop=True)
hospital_data_2021

,hospital_pk,Date,state,ccn,hospital_name,address,city,zip,hospital_subtype,countyFIPS,...,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum,Demo | State Geographic Boundaries
0,453311,2021-01-01,TX,453311,SHRINERS HOSPITALS FOR CHILDREN,815 MARKET STREET,GALVESTON,77550,Childrens Hospitals,48167,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,44.0
1,390223,2021-01-01,PA,390223,PENN PRESBYTERIAN MEDICAL CENTER,51 NORTH 39TH STREET,PHILADELPHIA,19104,Short Term,42101,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,39.0
2,390220,2021-01-01,PA,390220,JENNERSVILLE HOSPITAL,1015 WEST BALTIMORE PIKE,WEST GROVE,19390,Short Term,42029,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,39.0
3,030121,2021-01-01,AZ,030121,"MOUNTAIN VISTA MEDICAL CENTER, LP",1301 SOUTH CRISMON ROAD,MESA,85209,Short Term,4013,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,3.0
4,042008,2021-01-01,AR,042008,ARKANSAS EXTENDED CARE HOSPITAL-FORT SMITH,"7301 ROGERS AVENUE, # 4",FORT SMITH,72903,Long Term,5131,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265305,380033,2021-12-31,OR,380033,SACRED HEART UNIVERSITY DISTRICT,1255 HILYARD STREET,EUGENE,97401,Short Term,41039,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,38.0
265306,360027,2021-12-31,OH,360027,AKRON GENERAL MEDICAL CENTER,1 AKRON GENERAL AVENUE,AKRON,44307,Short Term,39153,...,NaN,0,NaN,-999999.0,1,-999999.0,17.0,1,17.0,36.0
265307,351312,2021-12-31,ND,351312,NORTHWOOD DEACONESS HEALTH CENTER,PO BOX 190,NORTHWOOD,58267,Critical Access Hospitals,38035,...,NaN,0,NaN,0.0,2,0.0,0.0,2,0.0,NaN
265308,370227,2021-12-31,OK,370227,ASCENSION ST JOHN OWASSO,12451 EAST 100TH STREET NORTH,OWASSO,74055,Short Term,40143,...,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0,37.0


In [5]:
#Merging the covid and hospital data on countyFIPS and Date column.
combined = pd.merge(hospital_data_2021,covid_data_2021_weekly, on=('countyFIPS','Date'))
combined

,hospital_pk,Date,state,ccn,hospital_name,address,city,zip,hospital_subtype,countyFIPS,...,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum,Demo | State Geographic Boundaries,County Name,StateFIPS,State,population,Number of Cases,Number of Deaths
0,453311,2021-01-01,TX,453311,SHRINERS HOSPITALS FOR CHILDREN,815 MARKET STREET,GALVESTON,77550,Childrens Hospitals,48167,...,NaN,0,NaN,44.0,Galveston County,48,TX,342139,158739,1706
1,450018,2021-01-01,TX,450018,UNIVERSITY OF TEXAS MEDICAL BRANCH,301 UNIVERSITY BOULEVARD,GALVESTON,77555,Short Term,48167,...,NaN,0,NaN,44.0,Galveston County,48,TX,342139,158739,1706
2,390223,2021-01-01,PA,390223,PENN PRESBYTERIAN MEDICAL CENTER,51 NORTH 39TH STREET,PHILADELPHIA,19104,Short Term,42101,...,0.0,7,0.0,39.0,Philadelphia County,42,PA,1584064,635257,17720
3,392050,2021-01-01,PA,392050,GOOD SHEPHERD PENN PARTNERS,1800 LOMBARD STREET,PHILADELPHIA,19146,Long Term,42101,...,0.0,7,0.0,39.0,Philadelphia County,42,PA,1584064,635257,17720
4,390204,2021-01-01,PA,390204,NAZARETH HOSPITAL,2601 HOLME AVE,PHILADELPHIA,19152,Short Term,42101,...,0.0,7,0.0,39.0,Philadelphia County,42,PA,1584064,635257,17720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256935,271323,2021-12-24,MT,271323,CLARK FORK VALLEY HOSPITAL,10 KRUGER RD,PLAINS,59859,Critical Access Hospitals,30089,...,0.0,4,0.0,27.0,Sanders County,30,MT,12113,11332,280
256936,450475,2021-12-24,TX,450475,UT HEALTH EAST TEXAS HENDERSON HOSPITAL,300 WILSON STREET,HENDERSON,75652,Short Term,48401,...,0.0,7,0.0,44.0,Rusk County,48,TX,54406,58804,1337
256937,040039,2021-12-24,AR,040039,ARKANSAS METHODIST MEDICAL CENTER,900 WEST KINGSHIGHWAY,PARAGOULD,72450,Short Term,5055,...,0.0,7,0.0,4.0,Greene County,5,AR,45325,76691,927
256938,161310,2021-12-24,IA,161310,ADAIR COUNTY MEMORIAL HOSPITAL,609 SE KENT,GREENFIELD,50849,Critical Access Hospitals,19001,...,0.0,7,0.0,16.0,Adair County,19,IA,7152,9201,304
